In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: canaanbreiss1 (lm-informants). Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd 
import wandb
import numpy as np

api = wandb.Api(timeout=50)
entity, project = "lm-informants", "lm-informants_maybefinal_three"  # set to your entity and project 
runs = api.runs(entity + "/" + project) 

In [ ]:
df = pd.DataFrame()

In [4]:
from tqdm import tqdm

summary_list, config_list, name_list = [], [], []
mega_df = pd.DataFrame()

all_data = []

for run in tqdm(runs): 
    
    # add all the keys that are logged that you want to download
    keys = [
        "step",
        "auc",
        ]
                
    df = pd.DataFrame()
    
    history = run.scan_history()
    
    history_df = pd.DataFrame(history)
                
    # filter ones that were killed
    if run.state != "finished":
        print("filtering run: ", run.path)
        continue
        
    # get the experiment config
    config = {k: v for k,v in run.config.items()
         if not k.startswith('_')}
    
    exp = {}
    
    # if key doesn't exist, set to nan (though this shouldn't happen after filtering empty runs)
    exp.update({f"{col}": history_df[~history_df[col].isnull()][col].values if col in history_df.columns else np.nan for col in keys})

    num_steps = len(exp[keys[0]])
    for k in keys:
        assert len(exp[k]) == num_steps
        
    # convert dict of lists to list of dicts
    results = [dict(zip(exp,t)) for t in zip(*exp.values())]
    
    for r in results:
        r.update({f"config/{key}": val for key, val in config.items()})
        r.update({f"strategy": run.name})
        r.update({f"wandb_id": run.path})
        
    all_data.extend(results)
    
    
# get status
    
mega_df = pd.DataFrame(all_data)


 53%|████████████████████████████████                            | 460/861 [04:08<04:16,  1.56it/s]

filtering run:  ['lm-informants', 'lm-informants_maybefinal_three', '3pa1aook']


100%|████████████████████████████████████████████████████████████| 861/861 [08:04<00:00,  1.78it/s]


In [7]:
display(mega_df)

,step,auc,config/run,config/n_init,config/strategy,config/tolerance,config/prior_prob,config/warm_start,config/feature_type,config/lexicon_file,config/converge_type,config/log_log_alpha_ratio,strategy,wandb_id
0,0.0,0.752471,9,0,kl_train_history,0.001,0.1,False,atr_harmony,None,symmetric,2,kl_train_history,"[lm-informants, lm-informants_maybefinal_three..."
1,1.0,0.752471,9,0,kl_train_history,0.001,0.1,False,atr_harmony,None,symmetric,2,kl_train_history,"[lm-informants, lm-informants_maybefinal_three..."
2,2.0,0.757603,9,0,kl_train_history,0.001,0.1,False,atr_harmony,None,symmetric,2,kl_train_history,"[lm-informants, lm-informants_maybefinal_three..."
3,3.0,0.759514,9,0,kl_train_history,0.001,0.1,False,atr_harmony,None,symmetric,2,kl_train_history,"[lm-informants, lm-informants_maybefinal_three..."
4,4.0,0.761605,9,0,kl_train_history,0.001,0.1,False,atr_harmony,None,symmetric,2,kl_train_history,"[lm-informants, lm-informants_maybefinal_three..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103594,145.0,0.784158,0,0,kl,0.001,0.5,NaN,atr_harmony,None,symmetric,4,kl,"[lm-informants, lm-informants_maybefinal_three..."
103595,146.0,0.784158,0,0,kl,0.001,0.5,NaN,atr_harmony,None,symmetric,4,kl,"[lm-informants, lm-informants_maybefinal_three..."
103596,147.0,0.784162,0,0,kl,0.001,0.5,NaN,atr_harmony,None,symmetric,4,kl,"[lm-informants, lm-informants_maybefinal_three..."
103597,148.0,0.783930,0,0,kl,0.001,0.5,NaN,atr_harmony,None,symmetric,4,kl,"[lm-informants, lm-informants_maybefinal_three..."


In [6]:
out_path = './runs.csv'
mega_df.to_csv(out_path)